#  PLEASE RUN THIS NOTEBOOK FOR STREAMLIT FILE IN GOOGLE COLLAB 

In [1]:
!pip install -q streamlit

!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [2]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.4 MB/s eta 0:00:00


# STREAMLIT

In [15]:
pip install streamlit pdfplumber pytesseract pillow transformers langchain faiss-cpu


In [ ]:
%%writefile app1.py
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import streamlit as st
import pdfplumber
import pytesseract
from PIL import Image
import re
import torch

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# ---------- Streamlit Setup ----------
st.set_page_config(page_title="📘 RAG SYSTEM WITH AGENTIC AI", layout="wide")

st.markdown("""
    <style>
        html, body, [class*="css"]  {
            font-family: 'Segoe UI', sans-serif;
            background-color: #F7F9FB;
        }

        .title-container {
            text-align: center;
            margin-bottom: 2rem;
        }

        .title-container h1 {
            font-size: 3rem;
            font-weight: 700;
            color: #0D47A1;
        }

        .title-container p {
            font-size: 1.2rem;
            color: #37474F;
            margin-top: -10px;
        }

        .section {
            background-color: #FFFFFF;
            padding: 2rem;
            border-radius: 16px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.05);
            margin-bottom: 2rem;
        }

        .stButton>button {
            background-color: #1565C0;
            color: white;
            font-weight: bold;
            border-radius: 8px;
            padding: 0.5rem 1rem;
        }

        .stButton>button:hover {
            background-color: #0D47A1;
        }

        input[type="text"] {
            border-radius: 8px;
            margin: 0 auto;
            display: block;
        }

        .output-section {
            background-color: #E3F2FD;
            padding: 1.5rem;
            border-left: 5px solid #2196F3;
            border-radius: 12px;
            margin-top: 1rem;
        }

        .output-section h4 {
            color: #0D47A1;
            margin-bottom: 0.5rem;
        }

        .block-container {
            padding: 1rem 2rem 2rem;
        }

        .upload-section, .download-section {
            margin-bottom: 1rem;
        }

        .custom-subheader {
            font-size: 1.4rem;
            font-weight: 600;
            color: #0D47A1;
            margin-top: 2rem;
            margin-bottom: 1rem;
        }
    </style>
""", unsafe_allow_html=True)


st.markdown("""
<div class="title-container">
    <h1>📘 RAG SYSTEM WITH AGENTIC AI</h1>
    <p>Build Smart QA using Retrieval-Augmented Generation and Autonomous Agents</p>
</div>
""", unsafe_allow_html=True)

uploaded_file = st.file_uploader("📄 Upload PDF or Image (for indexing)", type=['pdf', 'png', 'jpg', 'jpeg'])

raw_text = ""

if uploaded_file is not None:
    file_ext = uploaded_file.name.split(".")[-1].lower()
    if file_ext == "pdf":
        with pdfplumber.open(uploaded_file) as pdf:
            for page in pdf.pages:
                raw_text += page.extract_text() + "\n"
    elif file_ext in ["png", "jpg", "jpeg"]:
        image = Image.open(uploaded_file)
        raw_text = pytesseract.image_to_string(image)

    st.subheader("📄 Extracted Text from Uploaded File")
    st.write(raw_text[:1000] + "..." if len(raw_text) > 1000 else raw_text)

    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_text(raw_text)

    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(chunks, embedding_model)

    st.subheader("🔹 Text Chunks and Embeddings")
    for i, chunk in enumerate(chunks[:5]):
        with st.expander(f"Chunk {i+1}"):
            st.write(chunk)

    vectorstore.save_local("faiss_index")
    docsearch = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
    st.success("✅ FAISS index saved and loaded successfully!")

    model_id = "google/flan-t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

    generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer,
                         device=0 if torch.cuda.is_available() else -1, max_new_tokens=256)

    llm = HuggingFacePipeline(pipeline=generator)

    custom_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
You are a helpful assistant. Based on the context below, answer the question in 5 to 6 meaningful lines.

Context:
{context}

Question:
{question}

Helpful Answer:
"""
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": custom_prompt}
    )

    if "query_history" not in st.session_state:
        st.session_state.query_history = []

    col1, col2 = st.columns([1, 1])

    with col1:
        st.markdown('<div class="custom-subheader">💬 Ask a Question Manually</div>', unsafe_allow_html=True)
        query = st.text_input(
            "Type or reuse a previous query:",
            value="",
            placeholder="Start typing...",
            key="query_box"
        )

        # Suggest from previous queries while typing
        suggestions = [q for q in st.session_state.query_history if query.lower() in q.lower() and q != query]
        if suggestions:
            st.caption("Suggestions:")
            for suggestion in suggestions[:5]:
                if st.button(suggestion):
                    st.session_state.query_box = suggestion
                    query = suggestion

    with col2:
        st.markdown('<div class="custom-subheader">🖼️ OCR-Based Image Upload for Question</div>', unsafe_allow_html=True)
        query_image = st.file_uploader("Upload an image", type=['png', 'jpg', 'jpeg'], label_visibility="collapsed")

    if query_image is not None:
        image = Image.open(query_image)
        ocr_text = pytesseract.image_to_string(image)
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', ocr_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        st.info("🧾 Extracted Query from OCR:")
        st.code(cleaned_text)
        query = cleaned_text

    if st.button("Generate Answer") and query:
        result = qa_chain.run(query)

        if query not in st.session_state.query_history:
            st.session_state.query_history.append(query)

        st.markdown('<div class="custom-subheader">📄 Full RAG Output</div>', unsafe_allow_html=True)
        st.write(result)

        match = re.search(r'Helpful Answer:\s*(.*)', result, re.IGNORECASE | re.DOTALL)
        if match:
            final_answer = match.group(1).strip()
            st.subheader("📝 Final Answer (Extracted)")
            st.success(final_answer)
        else:
            st.warning("⚠️ Could not extract a structured 'Helpful Answer'. See the full result above.")


Overwriting app1.py


In [ ]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual ngrok authtoken
ngrok.set_auth_token("2xmU2eOYb4MJmlTirQ5GBUtX7qT_MXNsMtZGeWPKGZ2eJzi") #kindly set with your authtoker

# Run the Streamlit app in the background
!streamlit run app1.py &>/dev/null&

# Create a public URL using ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"Streamlit app is running at {public_url}")
except Exception as e:
    print(f"Error: {e}")
    print("Trying to run with localtunnel")
    !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501

Streamlit app is running at NgrokTunnel: "https://2e6d-34-105-86-142.ngrok-free.app" -> "http://localhost:8501"
